In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install gensim

In [3]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags
import numpy as np
import networkx as nx
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm
from nltk.tokenize import sent_tokenize

In [4]:
def embed_documents(docs) -> list:
  model = SentenceTransformer('all-mpnet-base-v2')
  return model.encode(docs)


In [5]:
def compute_cosine(A: list, B: list) -> float:
  return np.dot(A,B)/(norm(A)*norm(B))

In [6]:
def generate_summary(text, similarity_weight = 1, position_weight = 10, num_sentences = 10):
    sentences = sent_tokenize(text)
    clean_sentences = [' '.join(simple_preprocess(strip_tags(sent), deacc=True)) for sent in sentences]
    sentence_vectors = embed_documents(clean_sentences)

    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentence_vectors)):
        position_score = position_weight*(1.0 - (i / len(sentence_vectors)))
        for j in range(len(sentence_vectors)):
            if i != j:
                similarity_matrix[i][j] = similarity_weight*(compute_cosine(sentence_vectors[i], sentence_vectors[j])) + position_score


    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary_sentences = [s for _, s in ranked_sentences[:num_sentences]]

    summary = ' '.join(summary_sentences)

    return summary

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
text = '''
Kratos is a fictional character in the God of War series, a video game franchise developed by Santa Monica Studio and published by Sony Interactive Entertainment. Kratos, a Spartan warrior, serves as the protagonist throughout most of the series. His story is one of tragedy, revenge, and redemption, set in the world of Greek mythology.

The story of Kratos begins with his origins as a mortal warrior in Sparta. He was a skilled and ruthless fighter, eventually rising to become the captain of Sparta's army due to his unmatched prowess in battle. However, Kratos's life took a dark turn when he made a deal with Ares, the Greek god of war. In exchange for power, Kratos pledged his allegiance to Ares, becoming a ruthless and bloodthirsty servant of the god.

Driven by blind rage and fueled by the atrocities he committed under Ares's command, Kratos eventually sought to break free from his servitude. In a moment of clarity, he turned against his master, leading to a brutal conflict that culminated in Kratos killing Ares.

However, this victory did not bring peace to Kratos. Haunted by the memories of his past and plagued by nightmares, he sought to escape the consequences of his actions. In his quest for redemption, Kratos embarked on a journey to the highest peak of Mount Olympus, seeking forgiveness from the gods.

Throughout the series, Kratos faces numerous challenges and battles against various mythical creatures, gods, and titans. His journey takes him across the vast expanse of the Greek world, from the depths of the Underworld to the heights of Olympus itself.

As the series progresses, Kratos's story evolves, delving deeper into themes of family, sacrifice, and the consequences of one's actions. He grapples with his past, his relationships with his family, including his wife, Lysandra, and daughter, Calliope, and the burden of his sins.

In later installments of the series, such as "God of War" (2018), Kratos's story continues in the realm of Norse mythology, where he faces new challenges and confronts his own inner demons while attempting to forge a new path for himself and his son, Atreus.

Overall, the story of Kratos is one of redemption and self-discovery, as he struggles to break free from his past and find a semblance of peace in a world torn apart by gods and monsters.
'''

In [9]:
generate_summary(text, num_sentences = 3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'Kratos, a Spartan warrior, serves as the protagonist throughout most of the series. The story of Kratos begins with his origins as a mortal warrior in Sparta. His story is one of tragedy, revenge, and redemption, set in the world of Greek mythology.'